In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
# sys.path.append("jukebox_clone")
# sys.path.append("lucent_clone")
import jukebox
import torch
import librosa
import soundfile as sf
import scipy
import os
import numpy as np
import nussl
import matplotlib.pyplot as plt

from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
from jukebox.utils.jukebox_utils import get_forward_calls_encoder, split_model
rank, local_rank, device = setup_dist_from_mpi()

Using cuda True


In [2]:
# Load the model
model = "5b" # or "1b_lyrics"     
vqvae, *priors = MODELS[model]
hparams = setup_hparams(vqvae, dict(sample_length = 1048576))
vqvae = make_vqvae(hparams, device)
vqvae = vqvae.eval()

Running  wget -O /home/pajouheshgar/.cache/jukebox/models/5b/vqvae.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar


--2022-06-04 12:52:18--  https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.60, 13.107.213.60, 2620:1ec:bdf::60, ...
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7726329 (7.4M) [application/x-tar]
Saving to: ‘/home/pajouheshgar/.cache/jukebox/models/5b/vqvae.pth.tar’

     0K .......... .......... .......... .......... ..........  0% 14.5M 1s
    50K .......... .......... .......... .......... ..........  1% 7.75M 1s
   100K .......... .......... .......... .......... ..........  1% 8.05M 1s
   150K .......... .......... .......... .......... ..........  2%  225M 1s
   200K .......... .......... .......... .......... ..........  3% 16.0M 1s
   250K .......... .......... .......... .......... ..........  3% 3.47M 1s
   300K .......... .......... .......... .......... ..........  4%

Restored from /home/pajouheshgar/.cache/jukebox/models/5b/vqvae.pth.tar
0: Loading vqvae in eval mode


In [3]:
num_seconds = 2
sample_rate = 44100
t = np.linspace(0, num_seconds, sample_rate * num_seconds)

In [117]:
# Choose the level of JukeBox and hook it
from lucent.optvis.render import hook_model
from functools import partial
from jukebox.utils.jukebox_utils import get_forward_calls_encoder, get_forward_calls_decoder, split_model, compose_funclist

# Functions to play and save audio
from jukebox.utils.jukebox_utils import compose_funclist
play = compose_funclist([
    lambda x: x.detach().cpu().numpy()[0,0],
    # lambda x: (x - x.min()) / (x.max() - x.min()),
    # lambda x: np.clip(x, -1.0, 1.0),
    lambda x: x / max(x.max(), -x.min()),
    lambda x: nussl.AudioSignal(audio_data_array=x, sample_rate=44100),
    lambda x: (x, x.embed_audio())
])

save = compose_funclist([
    lambda x: x.detach().cpu().numpy()[0,0],
    # lambda x: (x - x.min()) / (x.max() - x.min()),
    # lambda x: np.clip(x, -1.0, 1.0),
    lambda x: x / max(x.max(), -x.min()),
    lambda x: nussl.AudioSignal(audio_data_array=x, sample_rate=44100),
    lambda x: x
])

post_process = compose_funclist([
    lambda x: x.detach().cpu().numpy()[0,0],
    lambda x: x / max(x.max(), -x.min()),
])


level = 2
encoder = vqvae.encoders[level]
bottleneck = vqvae.bottleneck.level_blocks[level]
decoder = vqvae.decoders[level]

encoder_hook, encoder_layers = hook_model(encoder, include_class_name=False)
encoder_calls, encoder_layer_names = get_forward_calls_encoder(encoder, prefix="")
decoder_hook, decoder_layers = hook_model(decoder, include_class_name=False)
decoder_calls, decoder_layer_names = get_forward_calls_decoder(decoder, prefix="")

In [79]:
# Split Encoder and Decoder
enc_layer_index = -1
l_enc = encoder_layer_names[enc_layer_index]
dec_layer_index = -1
l_dec = decoder_layer_names[dec_layer_index]

pre_enc, post_enc = split_model(encoder, l_enc, partial(get_forward_calls_encoder, prefix=""))
pre_dec, post_dec = split_model(decoder, l_dec, partial(get_forward_calls_decoder, prefix=""))

# Merge Encoder and Decoder
pre = compose_funclist([
    pre_enc,
    # post_enc,
    # lambda x: bottleneck(x)[1],
    # pre_dec,
    # post_dec,
])

post = compose_funclist([
    # pre_enc,
    post_enc,
    lambda x: bottleneck(x)[1],
    pre_dec,
    post_dec,
])


In [80]:
def calc_mean_std(features):
    """
    :param features: shape of features -> [batch_size, c, T]
    :return: features_mean, feature_s: shape of mean/std ->[batch_size, c, 1]
    """

    batch_size, c = features.size()[:2]
    features_mean = features.mean(dim=-1, keepdim=True)
    features_std = features.std(dim=-1, keepdim=True) + 1e-6
    return features_mean, features_std

def adain_style_fusion(content_features, style_features):
    final_feature = content_features.clone()
    content_mean, content_std = calc_mean_std(content_features)
    style_mean, style_std = calc_mean_std(style_features)
    fused_feature = style_std * (content_features - content_mean) / content_std + style_mean
    
    return fused_feature


def get_gram_matrix(h, remove_mean=True):
    b, c, T = h.shape
    assert b == 1
    f = h[0]
    if remove_mean:
        f = f - f.mean(dim=1, keepdim=True)
    G = torch.matmul(f, f.permute(1, 0))
    G = G / T
    
    return G

def whitening_transform(h):
    eps = 1e-10
    b, c, T = h.shape
    assert b == 1
    
    h = h.to(torch.double)
    G = get_gram_matrix(h).to(torch.double)
    
    
    E, D, V = torch.linalg.svd(G, full_matrices=True)
    
    i = len(D)
    for j in range(len(D)):
        if D[j] < eps:
            i = j
            break
            
    
    Dt = D[:i]    
    Dt = Dt ** (-0.5)
    Dt = torch.diag(Dt)
    D = torch.diag(D)
    Et = E[:, :i]
    Vt = V[:i, :]
    A = Et @ Dt @ Et.T
    
    f = h.reshape(c, T)
    f = f - f.mean(dim=1, keepdim=True)
    wh = A @ f

        
    wh = wh.reshape(b, c, T)
    return wh

def coloring_transform(wh, h_style):
    b, c, T = h_style.shape
    assert b == 1
        
    h_style = h_style.to(torch.double)
    G_style = get_gram_matrix(h_style).to(torch.double)
    
    E, D, V = torch.linalg.svd(G_style, full_matrices=True)

    D = D ** (0.5)
    D = torch.diag(D)
    A = E @ D
    A = A @ E.T
    A = A
    
    b2, c2, T2 = wh.shape
    wh = wh.reshape(c2, T2)
    # wh = wh
    sh = A @ wh
    sh = sh + h_style.reshape(c, T).mean(dim=1, keepdim=True)
    sh = sh.reshape(b2, c2, T2)
    
    return sh    

In [118]:
def style_transfer(x_content, x_style, method='adain', output_file_name='tmp.mp3'):
    assert method in ['adain', 'wct']
    
    
    with torch.no_grad():
        h_content = pre(x_content)
        h_style = pre(x_style)
        
        if method == 'wct':
            h_fuse = whitening_transform(h_content)
            h_fuse = coloring_transform(h_fuse, h_style).to(torch.float32)
        elif method == 'adain':
            h_fuse = adain_style_fusion(h_content, h_style)
        else:
            pass
        
        x_fuse_hat = post(h_fuse)
        
    audio = save(x_fuse_hat)
    audio.write_audio_to_file(f"out/style_transfer/{output_file_name}")

In [119]:
# Load wave wiles
freq = 261.63

sin_wav, _ = np.sin(2 * np.pi * freq * t), sample_rate
piano_wav, _ = librosa.load('data/notes/piano-C4.wav', sr=sample_rate)
violin_wav, _ = librosa.load('data/notes/violin-C4.wav', sr=sample_rate)
flute_wav, _ = librosa.load('data/notes/flute-C4.wav', sr=sample_rate)
trumpet_wav, _ = librosa.load('data/notes/trumpet-C4.wav', sr=sample_rate)



def preprocess_wave(wav):
    wav = wav[:num_seconds * sample_rate]
    x = torch.Tensor(wav).to(device)[None,None,:] / max(wav.max(), -wav.min())
    return x

sin_x = preprocess_wave(sin_wav)
piano_x = preprocess_wave(piano_wav)
violin_x = preprocess_wave(violin_wav)
flute_x = preprocess_wave(flute_wav)
trumpet_x = preprocess_wave(trumpet_wav)

instruments = ['sin', 'piano', 'violin', 'flute', 'trumpet']
xs = [sin_x, piano_x, violin_x, flute_x, trumpet_x]
transfer_algorithms = ['adain', 'wct']



In [122]:
for method in transfer_algorithms:
    for content_x, content_instrument in zip(xs, instruments):
        audio = save(content_x)
        audio.write_audio_to_file(f"out/style_transfer/{content_instrument}.mp3")
        for style_x, style_instrument in zip(xs, instruments):
            style_transfer(content_x, style_x, method=method, output_file_name=f"{content_instrument}2{style_instrument} {method}.mp3")

In [37]:
# import scipy
# from lucent.optvis.render import hook_model
# from functools import partial
# from jukebox.utils.jukebox_utils import get_forward_calls_encoder, get_forward_calls_decoder, split_model

# #x = torch.zeros(1,1,44100)
# level = 2
# encoder = vqvae.encoders[level]
# bottleneck = vqvae.bottleneck.level_blocks[level]
# decoder = vqvae.decoders[level]


# wave = scipy.signal.chirp(t, f_start, num_seconds, f_end)
# x = scipy.signal.chirp(t, f_start, num_seconds, f_end)[None, None, :]
# x = torch.from_numpy(x).cuda().float()


# hook, layers = hook_model(encoder, include_class_name=False)
# print(list(layers.keys())[:10])
# x_z = encoder(x)[-1]
# encoder_calls, encoder_layer_names = get_forward_calls_encoder(encoder, prefix="")

# for l in encoder_layer_names:
#     pre, post = split_model(encoder, l, partial(get_forward_calls_encoder, prefix=""))
#     try:
#         h1 = hook(l)
#         h2 = pre(x)
#         print((h2 == h1).all().item())
#         if not (h2 == h1).all().item():
#             error = torch.abs(h1 - h2).mean()
#             print(f"Layer {l} failed with error = {error}")
#     except:
#             continue
      
    
# hook, layers = hook_model(decoder, include_class_name=False)
# _, xs_quantized, _, _ = bottleneck(x_z)
# decoder([xs_quantized], all_levels=False)
# decoder_calls, decoder_layer_names = get_forward_calls_decoder(decoder, prefix="")
# # print(decoder)
# for l in decoder_layer_names:
#     pre, post = split_model(decoder, l, partial(get_forward_calls_decoder, prefix=""))
#     try:
#         h1 = hook(l)
#         h2 = pre(xs_quantized)
#         print((h2 == h1).all().item())
#         if not (h2 == h1).all().item():
#             error = torch.abs(h1 - h2).mean()
#             print(f"Layer {l} failed with error = {error}")
        
#     except:
#         continue

NameError: name 'f_start' is not defined